Temporary results of Corporate Incom Tax distribution to tax-payers

In [1]:
import sys
import taxcalc
from taxcalc import *
import pandas as pd
import numpy as np
from numba import jit
import numba

In [2]:
records_x = Records("puf.csv")
records_y = Records("puf.csv")

In [3]:
policy_x = Policy()
policy_y = Policy()
behavior_y = Behavior()
calcX = Calculator(policy_x, records_x)
calcY = Calculator(policy_y, records_y, behavior=behavior_y)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [4]:
calcX.advance_to_year(2017)
calcY.advance_to_year(2017)
assert calcX.current_year == 2017
assert calcY.current_year == 2017

In [5]:
calcX.calc_all()

In [6]:
reform = {2017: {"_II_rt7": [0.496]}}
policy_y.implement_reform(reform)
# Demonstrate that Plan X and Plan Y calculators are indeed different. 
print(calcX.policy.II_rt7)
print(calcY.policy.II_rt7)

0.396
0.496


In [7]:
behavior_y.update_behavior({2017: {'_BE_inc': [-0.15], '_BE_sub': [0.4]}})
calcY_behavior1 = Behavior.response(calcX, calcY)
behavior_y.update_behavior({2017: {'_BE_inc': [-0.15], '_BE_sub': [0.5]}})
calcY_behavior2 = Behavior.response(calcX, calcY)
behavior_y.update_behavior({2017: {'_BE_inc': [0.0], '_BE_sub': [0.4]}})
calcY_behavior3 = Behavior.response(calcX, calcY)

In [8]:
# Demonstrate that taxpayers' income was affected by the behavioral effects.
print(calcY.records.e00200.sum())
print(calcY_behavior1.records.e00200.sum())
print(calcY_behavior2.records.e00200.sum())
print(calcY_behavior3.records.e00200.sum())

40681683225.0
38887827999.4
38373606744.6
38624798205.8


In [9]:
# Demonstrate that total tax was affected by the behavioral effects. 
print(calcX.records.combined.sum())
print(calcY.records.combined.sum())
print(calcY_behavior1.records.combined.sum())
print(calcY_behavior2.records.combined.sum())
print(calcY_behavior3.records.combined.sum())

43476646184.5
46402047946.2
43849701719.0
43185112636.7
43604367079.6


In [10]:
DistCorpIncTax(calcX.policy, calcX.records)

,share_corptax_burden
0,7.964186e+00
1,1.887646e+07
2,3.708569e+07
3,-2.695613e+05
4,-4.994644e+04
5,8.511275e+05
6,6.965848e+06
7,1.191294e+06
8,6.846493e+05
9,1.054586e+06


In [11]:
create_corpinctax_table(calcX, "weighted_deciles", "weighted_sum")

,s006,expanded_income,c00100,share_corptax_burden
0,"17,004,988","-119,449,251,743","-124,254,447,367","-21,010,701,485"
1,"17,004,489","152,378,742,352","104,382,534,735","-3,341,672,388"
2,"17,006,659","273,930,511,045","175,769,613,967","-1,887,380,105"
3,"17,004,285","400,560,764,101","271,255,931,008","-4,505,952,036"
4,"17,006,306","558,266,448,736","414,206,537,931","-4,527,702,496"
5,"17,005,711","754,024,219,820","606,882,342,664","-6,706,738,832"
6,"17,005,578","1,008,752,286,904","867,578,622,129","-8,918,376,001"
7,"17,004,128","1,379,302,477,096","1,261,792,218,778","-9,160,920,710"
8,"17,006,254","2,054,398,833,867","1,912,589,026,337","-8,773,837,161"
9,"17,006,552","6,272,407,565,304","5,953,769,770,507","168,833,281,214"


In [12]:
create_corpinctax_table(calcX, "weighted_deciles", "weighted_avg")

,expanded_income,s006,c00100,share_corptax_burden
0,"-7,024","17,004,988","-7,307","-1,236"
1,"8,961","17,004,489","6,139",-197
2,"16,107","17,006,659","10,335",-111
3,"23,556","17,004,285","15,952",-265
4,"32,827","17,006,306","24,356",-266
5,"44,339","17,005,711","35,687",-394
6,"59,319","17,005,578","51,017",-524
7,"81,116","17,004,128","74,205",-539
8,"120,803","17,006,254","112,464",-516
9,"368,823","17,006,552","350,087","9,928"


In [13]:
create_corpinctax_table(calcX, "small_income_bins", "weighted_sum")

,s006,expanded_income,c00100,share_corptax_burden
0,"4,766,539","-151,159,828,775","-151,251,969,357","-18,064,503,231"
1,"11,933,771","30,166,679,619","25,766,280,670","-2,785,768,489"
2,"11,222,794","84,841,559,555","63,763,953,735","-2,939,075,734"
3,"11,842,787","149,302,233,838","89,857,596,332","-1,384,770,753"
4,"12,624,678","220,790,072,087","147,819,537,147","-1,405,632,040"
5,"10,000,060","223,259,725,713","151,035,592,537","-2,299,522,141"
6,"8,942,973","246,732,831,091","170,444,698,692","-3,045,664,335"
7,"16,371,822","566,134,757,582","423,732,642,474","-4,626,417,503"
8,"13,283,431","596,999,684,070","484,254,671,843","-4,445,028,427"
9,"22,758,269","1,400,153,627,981","1,216,153,724,855","-11,692,533,205"


In [14]:
create_corpinctax_table(calcX, "large_income_bins", "weighted_sum")

,s006,expanded_income,c00100,share_corptax_burden
0,"4,766,539","-151,159,828,775","-151,251,969,357","-18,064,503,231"
1,"23,156,565","115,008,239,174","89,530,234,405","-5,724,844,223"
2,"24,467,465","370,092,305,925","237,677,133,479","-2,790,402,792"
3,"18,943,034","469,992,556,804","321,480,291,229","-5,345,186,476"
4,"16,371,822","566,134,757,582","423,732,642,474","-4,626,417,503"
5,"13,283,431","596,999,684,070","484,254,671,843","-4,445,028,427"
6,"22,758,269","1,400,153,627,981","1,216,153,724,855","-11,692,533,205"
7,"13,547,772","1,163,806,445,651","1,070,601,241,917","-8,671,679,461"
8,"23,023,733","3,181,461,751,494","2,964,155,988,738","-10,073,393,879"
9,"9,736,321","5,022,083,057,576","4,787,638,191,106","171,433,989,197"


In [15]:
create_corpinctax_table(calcX, "webapp_income_bins", "weighted_sum")

,s006,expanded_income,c00100,share_corptax_burden
0,"4,766,539","-151,159,828,775","-151,251,969,357","-18,064,503,231"
1,"23,156,565","115,008,239,174","89,530,234,405","-5,724,844,223"
2,"24,467,465","370,092,305,925","237,677,133,479","-2,790,402,792"
3,"18,943,034","469,992,556,804","321,480,291,229","-5,345,186,476"
4,"16,371,822","566,134,757,582","423,732,642,474","-4,626,417,503"
5,"13,283,431","596,999,684,070","484,254,671,843","-4,445,028,427"
6,"22,758,269","1,400,153,627,981","1,216,153,724,855","-11,692,533,205"
7,"13,547,772","1,163,806,445,651","1,070,601,241,917","-8,671,679,461"
8,"23,023,733","3,181,461,751,494","2,964,155,988,738","-10,073,393,879"
9,"7,922,248","2,275,472,665,060","2,147,876,336,172","5,829,596,514"
